In [1]:
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pyplot as plt

In [2]:
class PDF(object):
    def __init__(self, pdf, size=(200,200)):
        self.pdf = pdf
        self.size = size

    def _repr_html_(self):
        return '<iframe src={0} width={1[0]} height={1[1]}></iframe>'.format(self.pdf, self.size)

    def _repr_latex_(self):
        return r'\includegraphics[width=1.0\textwidth]{{{0}}}'.format(self.pdf)

In [3]:
!mkdir -p ./5.coup_delta_heatmap

In [4]:
os.chdir('/home/jlyu/res_lambda/Run_On_Lvjie/5.coup_delta_heatmap/')

Discussion with Hong

The peaks center not obvious on the delta heatmap, so we using the real center of the peaks;

Do not plot the peak region and just + - 5k

# Tester the reference point does not having the same outputs as the 1.2_peak.pdf

1_2.peak.pdf using the coup bed file to plot the heatmap, which has the phenomena that the signal spans across the whole region

while using the delta heatmap the results does not show such pattern

Correspondence:
The 1_2.peak.pdf have wide signal spanning across +- 5k due to the plot are stretched vertically and the color box becomes larger
Another reason is the bin size; small binsize having the small signal values

## 1.1 Use the union of the T1 WT to do the heatmap plot

In [19]:
ctrl = 'ENL'
treats = ['T1', 'T2', 'T3', 'Y78A']
# sigpath: store the previous generated merged peaks signal for T and wt
sigpath = './res_avg_signal/' 
# outpath = './res_delta_heatmap/'
outpath = './res_delta_heatmap/peak_center_no_flank_5k/'
bwpath = '/home/xc3/output/ENL2/ChIP/bw_nsp/'
exp = '293'
treat = treats[0]

In [38]:
    sigfile = f'{sigpath}293_{treat}_WT.merge.sig.anno.pval.txt'
    line = "awk \'BEGIN{OFS=\"\\t\"}{if (NR > 1) print $1, $2, $3,NR-2,\".\", \"+\"}\'" + f' {sigfile} > {outpath}{treat}.bed'
    print(line + '\n****')
    os.system(line)
    bw_files = [
            glob.glob(bwpath + exp + '*' + tmp + '*.bw')[0]
            for tmp in [treat, ctrl]
    ]
    bw_files_format = len(bw_files) * "{} "
    bw_files_format = bw_files_format.format(*bw_files)
    bed_file_format = f'{outpath}{treat}.bed'
    command = "computeMatrix reference-point --referencePoint center -b 5000 -a 5000 -bs 100 --missingDataAsZero -R {0} -S {1} --skipZeros -o {2}.scale.gz --outFileNameMatrix {3}.scale.txt ".format(
    bed_file_format, bw_files_format,
    outpath + exp + '_' + treat + '_' + ctrl,
    outpath + exp + '_' + treat + '_' + ctrl)
    print(command + '\n\n')
    os.system(command)
#     os.system(
#         'rm {}.scale.gz'.format(outpath + exp + '_' + treat + '_' + ctrl))     
    line = f'plotHeatmap --sortUsingSamples 1 -m {outpath}293_T1_ENL.scale.gz -out {outpath}test.peak.pdf \
    --outFileNameMatrix test.mat --colorMap Blues Blues  --dpi 300 --refPointLabel center'
    print(line)
    os.system(line)

awk 'BEGIN{OFS="\t"}{if (NR > 1) print $1, $2, $3,NR-2,".", "+"}' ./res_avg_signal/293_T1_WT.merge.sig.anno.pval.txt > ./res_delta_heatmap/peak_center_no_flank_5k/T1.bed
****
computeMatrix reference-point --referencePoint center -b 5000 -a 5000 -bs 100 --missingDataAsZero -R ./res_delta_heatmap/peak_center_no_flank_5k/T1.bed -S /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_T1_treat.nsp.bw /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_ENL_treat.nsp.bw  --skipZeros -o ./res_delta_heatmap/peak_center_no_flank_5k/293_T1_ENL.scale.gz --outFileNameMatrix ./res_delta_heatmap/peak_center_no_flank_5k/293_T1_ENL.scale.txt 


plotHeatmap --sortUsingSamples 1 -m ./res_delta_heatmap/peak_center_no_flank_5k/293_T1_ENL.scale.gz -out ./res_delta_heatmap/peak_center_no_flank_5k/test.peak.pdf --outFileNameMatrix test.mat --colorMap Blues Blues  --dpi 300 --refPointLabel center


0

# 3. The delta heatmap for 1.2 coup peaks

previous created delta heatmap by jiejun has wider signal spread
Another reason is that when we plot the tss region, lots of genes shared the same region, so the color box is huge

In [5]:
workPath = '/home/jlyu/xc3/experiment/ENL2/Chip-seq_analysis/no_spike_in/re_do_mapping_with_lambda/Run_On_Lvjie/4.coup_peaks/'
os.chdir(workPath)
!pwd

/home/jlyu/xc3/experiment/ENL2/Chip-seq_analysis/no_spike_in/re_do_mapping_with_lambda/Run_On_Lvjie/4.coup_peaks


In [67]:
#format the bed file into the bed file add three columns more; 
names = ['1_2']
for name in names:
    Bed = f'293.all.{name}.sig.anno.txt'
    outBed = f'293.all.{name}.new.heat.bed'
    line = "awk 'BEGIN{OFS=\"\\t\"}{if (NR>1){print $1,$2,$3,NR-2,\".\",\"+\"}}'" + f' {Bed} > {outBed}'
    print(line)
    os.system(line)

awk 'BEGIN{OFS="\t"}{if (NR>1){print $1,$2,$3,NR-2,".","+"}}' 293.all.1_2.sig.anno.txt > 293.all.1_2.new.heat.bed


## for each pair sample extract the binning signal

In [92]:
samples = treats + [ctrl]
for sample in samples:
    bwfile = glob.glob(bwpath + exp + '*' + sample + '*.bw')[0]
    outBed = "293.all.1_2.new.heat.bed"
    print(bwfile)
    line = f'~/software/deeptools/bin/computeMatrix reference-point --referencePoint center -S {bwfile} -R {outBed} --beforeRegionStartLength 5000 --afterRegionStartLength 5000 --binSize 100 --skipZeros --missingDataAsZero -o PeakMatrix_293_F_{sample}_co-UP_genes.gz -p 8'
    print(line + '\n****\n')
    os.system(line)

/home/xc3/output/ENL2/ChIP/bw_nsp/293_F_T1_treat.nsp.bw
~/software/deeptools/bin/computeMatrix reference-point --referencePoint center -S /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_T1_treat.nsp.bw -R 293.all.1_2.new.heat.bed --beforeRegionStartLength 5000 --afterRegionStartLength 5000 --binSize 100 --skipZeros --missingDataAsZero -o PeakMatrix_293_F_T1_co-UP_genes.gz -p 8
****

/home/xc3/output/ENL2/ChIP/bw_nsp/293_F_T2_treat.nsp.bw
~/software/deeptools/bin/computeMatrix reference-point --referencePoint center -S /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_T2_treat.nsp.bw -R 293.all.1_2.new.heat.bed --beforeRegionStartLength 5000 --afterRegionStartLength 5000 --binSize 100 --skipZeros --missingDataAsZero -o PeakMatrix_293_F_T2_co-UP_genes.gz -p 8
****

/home/xc3/output/ENL2/ChIP/bw_nsp/293_F_T3_treat.nsp.bw
~/software/deeptools/bin/computeMatrix reference-point --referencePoint center -S /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_T3_treat.nsp.bw -R 293.all.1_2.new.heat.bed --beforeRegionStartLen

chr15	96872571	96885658	0	.	+
chr7	5457629	5470124	1	.	+
chr6	1608510	1616027	2	.	+
chr17	79368489	79375603	3	.	+
chr12	12869276	12879696	4	.	+
chr11	65186288	65204652	5	.	+
chr7	27197889	27240775	6	.	+
chr11	65264641	65277345	7	.	+
chr18	19747704	19753749	8	.	+
chr6	32934633	32953286	9	.	+
/home/xc3/experiment/ENL2/Chip-seq_analysis/no_spike_in/re_do_mapping_with_lambda/res_heatmap_coup_peaks
